In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier

### IDEA
La idea de este notebook es probar solo con los datos relacionados al area.

In [ ]:
# En base a lo visto por el TP1 las siguientes areas son las mas
# postuladas
#areas_importantes = [
#    'Ventas', 'Administración', 'Producción', 'Comercial', 'Atención al Cliente',
#    'Recepcionista', 'Call Center', 'Telemarketing', 'Tesorería', 'Mantenimient y Limpieza',
#]

# 2.  AVISOS

In [ ]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

In [ ]:
# Solo me quedo con nombre de area
avisos = avisos[['idaviso', 'nombre_area', 
                 'nombre_zona', 'tipo_de_trabajo', 
                 'nivel_laboral']]
avisos.head()

In [ ]:
# Cargamos previamente los datos de postulaciones para encontrar las areas mas postuladas
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

In [ ]:
mas_postulados = pd.merge(postulaciones[['idaviso']], avisos[['idaviso', 'nombre_area']], on='idaviso', how='inner')
mas_postulados.head()

In [ ]:
areas_importantes = mas_postulados['nombre_area'].value_counts().head(10).index

In [ ]:
avisos.drop_duplicates(subset='idaviso', inplace=True)
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_importantes else 'Otra Area')

In [ ]:
# Uso one hot encoding para area
dummies_area = pd.get_dummies(avisos['nombre_area'])
avisos = pd.concat([avisos, dummies_area], axis=1)
avisos.drop(columns='nombre_area', inplace=True)
avisos.head()

In [ ]:
# ZONA
# 1 GBA
# 0 Capital y otros
avisos['nombre_zona'] = avisos['nombre_zona'].apply(lambda x: 1 if x == 'Gran Buenos Aires' else 0)

In [ ]:
# Tipo de trabajo
# Segun horas aprox de trabajo
# 9 Full-time
# 6 Partime
# 4 otros
horas_trabajo = {
    'Full-time':9,
    'Part-time':6,
}
avisos['tipo_de_trabajo'] = avisos['tipo_de_trabajo'].apply(lambda x: horas_trabajo[x] if x in horas_trabajo else 4)
avisos.head()

In [ ]:
avisos['nivel_laboral'].value_counts()

In [ ]:
# Nivel laboral
# 1 Senior
# 0 otros
nivel = {
    'Otro': 1, 
    'Junior': 2,
    'Senior / Semi-Senior': 4,                                                 
    'Jefe / Supervisor / Responsable': 8,         
    'Gerencia / Alta Gerencia / Dirección': 16
}
#avisos['nivel_laboral'] = avisos['nivel_laboral'].apply(lambda x: 1 if x == 'Senior / Semi-Senior' else 0)
avisos['nivel_laboral'] = avisos['nivel_laboral'].apply(lambda x: nivel[x] if x in nivel else 0)

In [ ]:
avisos.head()

# 2. VISTAS

In [ ]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])
vistas.head()

In [ ]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas = vistas[['idaviso', 'idpostulante']]
vistas.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
vistas.head()

In [ ]:
# Aniado la informacion del area de cada aviso
avisos_vistos = pd.merge(vistas, 
                         avisos.drop(columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral']),
                         on='idaviso', 
                         how='inner')
avisos_vistos.head()

In [ ]:
avisos_vistos.drop(columns='idaviso',inplace=True)

In [ ]:
# Cada columna representa la cantidad de avisos vistos por area para cada postulante
avisos_vistos = avisos_vistos.pivot_table(index='idpostulante', aggfunc='sum')
avisos_vistos.head()

In [ ]:
# Ningun usuario supera 2^16 cantidad de vistas por area
avisos_vistos = avisos_vistos.astype('uint16')

In [ ]:
# le agrego la palabra 'vistas' adelante para que sea mas claro y no se confunda luego con el area postulada
nuevos_nombres = []
for area in avisos_vistos.columns:
    area = "vistas " + area
    nuevos_nombres.append(area)
avisos_vistos.columns = nuevos_nombres
avisos_vistos.head()

# 3. GENERO Y EDAD

In [ ]:
genyedad = pd.concat([
    pd.read_csv("../input/datos/fiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_2_postulantes_genero_y_edad.csv")
])
genyedad.head()

In [ ]:
# elimino los datos repetidos
genyedad.drop_duplicates(subset='idpostulante', inplace=True)
genyedad.head()

In [ ]:
# 1 MASC
# 0 FEM + no declara
genyedad['sexo'] = genyedad['sexo'].apply(lambda x: 1 if x == 'MASC' else 0)

In [ ]:
# Calculo las edades
genyedad['fecha_dt'] = pd.to_datetime(genyedad['fechanacimiento'], errors='coerce')
genyedad.head()

In [ ]:
genyedad['edad'] = 2018 - genyedad['fecha_dt'].dt.year
genyedad.head()

In [ ]:
genyedad.drop(columns=['fechanacimiento', 'fecha_dt'], inplace=True)
genyedad.head()

In [ ]:
genyedad.fillna(genyedad['edad'].mean(), inplace=True)
genyedad = genyedad.astype({'sexo':'int8', 'edad':'uint8'})
genyedad.head()

In [ ]:
# lo uno con avisos_vistos
usuarios = pd.merge(avisos_vistos, genyedad, on='idpostulante', how='inner')
usuarios.head()

In [ ]:
# Con estos daatos caracterizamos a los usuarios

# 2.9 Caracteristicas cruzadas.

In [ ]:
avisos.head()

In [ ]:
vistas.head()

In [ ]:
# Busco que caracteristicas tienen los usuarios que se postulan

In [ ]:
usuarios.head()

In [ ]:
vistas_usuarios = pd.merge(vistas, usuarios[['idpostulante', 'sexo', 'edad']], on='idpostulante', how='left')
vistas_usuarios.head()

In [ ]:
vistas_usuarios.isnull().sum()

In [ ]:
vistas_usuarios['sexo'].mean()

In [ ]:
vistas_usuarios['sexo'] = vistas_usuarios['sexo'].fillna(random.getrandbits(1))

In [ ]:
vistas_usuarios['edad'] = vistas_usuarios['edad'].fillna(vistas_usuarios['edad'].mean())

In [ ]:
vis_us = vistas_usuarios.drop(columns='idpostulante').pivot_table(index='idaviso', aggfunc=np.nanmean)
vis_us.head()

In [ ]:
vis_us = vis_us.astype({'edad':'uint8'})

In [ ]:
vis_us.head()

In [ ]:
# Agrego la nueva informacion a los avisos.
avisos.head()

In [ ]:
avisos  = pd.merge(avisos, vis_us, on='idaviso', how='left')
avisos.head()

In [ ]:
avisos.isnull().sum()

In [ ]:
avisos['edad'] = avisos['edad'].fillna(avisos['edad'].mean())
avisos['sexo'] = avisos['sexo'].fillna(random.getrandbits(1))

In [ ]:
avisos.rename(columns={'edad':'prom_edad', 'sexo':'sexo_may'}, inplace=True)
avisos.head()

# 3. POSTULACIONES

In [ ]:
# Cargamos los datos antes para obtener las areas mas postuladas
#postulaciones = pd.concat([
#    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
#    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
#])
#postulaciones.head()

In [ ]:

# La carga de datos se hizo anteriormente
postulaciones.drop(columns='fechapostulacion', inplace=True)
postulaciones.head()

In [ ]:
postulaciones.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)

In [ ]:
postulaciones.head()

In [ ]:
# Busco los avisos y las areas de cada postulacion
postulaciones_detalle = pd.merge(postulaciones, avisos, on='idaviso', how='inner')

In [ ]:
# 1 en el area de la categoria
postulaciones_detalle.head()

In [ ]:
# Finalmente uno la informacion de los usuarios y sus postulaciones
datos = pd.merge(usuarios, postulaciones_detalle, on='idpostulante', how='inner')
datos.head()

In [ ]:
# Todos estos avisos fueron postulaciones y por lo tanto se clasifican como postulados
# POSTULADO 1
# NO_POSTULADO 0
datos['postulado'] = 1
datos.head()

In [ ]:
datos.info()

# 4. Generacion de NO_POSTULACIONES

In [ ]:
# En realidad es la info que describe a los usuarios segun sus vistas.
# TODO cambiar el nombre de avisos_vistos a usuarios
usuarios.head()

In [ ]:
# Categoria de cada aviso
avisos.head()

In [ ]:
# Mezclo al azar usuarios con sus vistas mas algun aviso cualquiera.
N = 5000000
usuarios_random = usuarios.sample(n=N, replace=True, random_state=44)
avisos_random = avisos.sample(n=N, replace=True, random_state=99)

In [ ]:
usuarios_random.reset_index(inplace=True)
avisos_random.reset_index(inplace=True)
usuarios_random.drop(columns='index', inplace=True)
avisos_random.drop(columns='index', inplace=True)

In [ ]:
avisos_random.size

In [ ]:
# No se porque tienen tamanios distintos asi que solo tomo los primeros N de 
# avisos_random
datos_random = pd.concat([usuarios_random, avisos_random], axis=1)

In [ ]:
datos_random.head()

In [ ]:
# POSTULADO 1
# NO_POSTULADO 0
datos_random['postulado'] = 0
datos_random.head()

In [ ]:
# Estos ultimos son los datos no postulados
datos_random.drop_duplicates(subset=['idpostulante', 'idaviso'],inplace=True)
datos_random.head()

# DATOS FINALES

In [ ]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")
# Recolecto su informacion en los distintos dataFrames

# Agrego datos de usuario
final = pd.merge(final100k, usuarios, on='idpostulante', how='left')
final.fillna(0, inplace=True)

# agrego datos de avisos
final = pd.merge(final, avisos, on='idaviso', how='left')
final.fillna(0, inplace=True)
X_final = final.drop(columns=['id', 'idaviso', 'idpostulante']) 
X_final.head()

# 5. Datos de entrenamiento para el algoritmo

In [ ]:
# Me creo un DF con los datos postulados y no postulados
df = pd.concat([datos,datos_random])
df.drop_duplicates(subset=['idpostulante', 'idaviso'], inplace=True)
df.head()

In [ ]:
# Voy a tomar tamanios mas pequenios de todos los datos disponibles
# Cambiar nombre despues
df_original = df

In [ ]:
# pruebo con la mitad
df = df_original.sample(frac=0.5)

In [ ]:
# X Datos
# y Categorias
X = df.drop(columns=['idpostulante', 'idaviso', 'postulado'])
# adicionales
X.head()

In [ ]:
y = df['postulado']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [ ]:
#for i in range(-1, 10):
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

# 7. PREDICCION

In [ ]:
y_final = clf.predict(X_final)

In [ ]:
y_final

# 8. SUMMIT

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
y_summit.head()

In [ ]:
y_summit.info()

In [ ]:
summit = y_summit.reset_index()
summit.head()

In [ ]:
summit.rename(columns={'index':'id'}, inplace=True)
summit.head()

In [ ]:
# Guardo los datos
summit.to_csv("summitGSNB.csv", index=False)
